### Bibliotecas

In [1]:
# tratamento de dados
import pandas as pd
import numpy as np

# web scraping
import requests
from bs4 import BeautifulSoup

In [2]:
# instanciando url e conectando
url = 'http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view'
response = requests.get(url)

# verificando se a conexao deu certo e extraindo as informaçoes da tabela com os dados
if response.status_code == 200:
    print('Requisição bem sucedida')

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'grd_DXMainTable'})
    rows = table.find_all('tr')

    data_list = []
    price_list = []

    for row in rows:
        columns = row.find_all('td', {'class': 'dxgv'})

        if len(columns) == 2:
            data = columns[0].text.strip()
            price = columns[1].text.strip()

            data_list.append(data)
            price_list.append(price)

    df = pd.DataFrame({
                        'data': data_list,
                        'preco': price_list
                    })

else:
    print('Problema na requisição')

Requisição bem sucedida


In [3]:
df.head()

,data,preco
0,18/11/2024,"74,35"
1,15/11/2024,"73,45"
2,14/11/2024,"73,39"
3,13/11/2024,"72,86"
4,12/11/2024,"72,56"


### Tratamento dos dados

In [4]:
# criando copia para trabalhar com os dados
dados_preco_petroleo = df.copy()

In [5]:
# alterando o tipo de dados
dados_preco_petroleo = dados_preco_petroleo.assign(
                                                    data = pd.to_datetime(dados_preco_petroleo['data'], dayfirst=True),
                                                    preco = dados_preco_petroleo['preco'].str.replace(',', '.').astype('float')
)

In [6]:
dados_preco_petroleo.head(3)

,data,preco
0,2024-11-18,74.35
1,2024-11-15,73.45
2,2024-11-14,73.39


In [7]:
print(f'Número de linhas: {dados_preco_petroleo.shape[0]}')
print(f'Número de colunas: {dados_preco_petroleo.shape[1]}')

print('                                              ')

print(f'Data mínima: {dados_preco_petroleo.data.min()}')
print(f'Data máxima: {dados_preco_petroleo.data.max()}')

Número de linhas: 11302
Número de colunas: 2
                                              
Data mínima: 1987-05-20 00:00:00
Data máxima: 2024-11-18 00:00:00


In [8]:
dados_preco_petroleo.to_csv(r'D:\Documentos\FIAP - Pos Tech Data Analytics\Tech Challeng - Preços Petróleo\Tech_Challeng_FIAP_Fase_04\Dados\dados_petroleo.csv', index=False)